In [ ]:
!pip install gspread oauth2client

In [ ]:
!pip install pygsheets

In [ ]:
!pip install pyairtable

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
import pandas as pd


In [ ]:
def get_rankings(participant_name):
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
    client = gspread.authorize(creds)
    sheet = client.open(f"2022_mm_{participant_name}").get_worksheet(0)
    player_ids = sheet.col_values(1)
    ranks = sheet.col_values(2)
    rankings = [None] * 961
    for idx, rank in enumerate(ranks):
      if rank != '' and rank != 'rank':
        rankings[int(rank)] = player_ids[idx]
    return list(filter(None, rankings))

In [ ]:
def populate_rankings(participant_list):
  rank_dict = {}
  for participant in participant_list:
    rank_dict[participant] = get_rankings(participant)
  return rank_dict

In [ ]:
def make_pick(participant, rank_dict, drafted_dict):
  for idx, pick in enumerate(rank_dict[participant]):
    if drafted_dict[pick] == False:
      drafted_dict[pick] = True
      rank_dict[participant] = rank_dict[participant][idx:]
      return pick

In [ ]:
def get_player_ids():
  scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
  creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
  client = gspread.authorize(creds)
  sheet = client.open(f"2022_mm_template").get_worksheet(0)
  player_ids = sheet.col_values(1)
  return player_ids


In [ ]:
def create_drafted_dict(ids):
  drafted_dict = {}
  for id in ids[1:]:
    drafted_dict[id] = False 
  return drafted_dict

In [ ]:
player_ids = get_player_ids()

In [ ]:
drafted_dict = create_drafted_dict(player_ids)

In [ ]:
participants = ['rachel','wes','mitchell','travis','kirk','trevor','colin','jordan','landon','ross','barry','joe']

In [ ]:
def itemize_pick(pick, round, participant, player):
  return {"pick": pick, "round": round, "participant": participant, "player": player}

  

In [ ]:
def draft(participants, rank_dict, drafted_dict, player_dict):
  pick = 1
  results = []
  for idx in range(6):
    round = idx * 2 + 1
    for participant_turn in participants:
      player = make_pick(participant_turn, rank_dict, drafted_dict)
      pick_object = itemize_pick(pick, round, participant_turn, player)
      results.append(pick_object)
      player_dict[player] = pick_object
      pick += 1
    
    round += 1
    for participant_turn in reversed(participants):
      player = make_pick(participant_turn, rank_dict, drafted_dict)
      pick_object = itemize_pick(pick, round, participant_turn, player)
      results.append(pick_object)
      player_dict[player] = pick_object
      pick += 1

  return results



In [ ]:
rank_dict = populate_rankings(participants)


In [ ]:
player_dict = {}
draft_results = draft(participants, rank_dict, drafted_dict, player_dict)

In [ ]:
pd.DataFrame(draft_results).to_csv("final_draft.csv")

In [ ]:
AIRTABLE_TOKEN="keyl9LrLLdDp8h8i6"
AIRTABLE_BASE_ID="app8SX5ie2AsXBerJ"
PLAYERS_TABLE_ID = "tblYbItXYdXMNzfwc"
TEST_TABLE_ID = "tbl0Yv0I6nQcgrPCv"

In [ ]:
AIRTABLE_URL = f"https://api.airtable.com/v0/{AIRTABLE_BASE_ID}"

In [ ]:
from pyairtable import Table

In [ ]:
table = Table(AIRTABLE_TOKEN, AIRTABLE_BASE_ID, PLAYERS_TABLE_ID)

In [ ]:
def get_record_id(table, player_name):
  record = table.first(formula=f"{{player_id}}='{player_name}'")
  return record["id"]

In [ ]:
def update_draft_pick(table, id, pick_obj):
  table.update(id, {"participant": pick_obj["participant"], "pick": pick_obj["pick"], "rd": str(pick_obj["round"])})
  

In [ ]:
def record_draft_in_airtable(player_dict, table):
  for player in player_dict:
    id = get_record_id(table, player)
    pick_obj = player_dict[player]
    update_draft_pick(table, id, pick_obj)
    print(f"Updated {player} to {pick_obj['participant']}, round {pick_obj['round']}, pick {pick_obj['pick']}", )


In [ ]:
record_draft_in_airtable(player_dict, table)